# <span style="color:darkred">Multiplicative Schwarz  using multiple strategies</span>.

<img align="left" src="Schwarz.jpg" width=120 height=120 />

### [Simon Tavener](https://www.math.colostate.edu/~tavener/) and Andre Leautaud

#### Full text for this notebook can be found at the following [link](https://www.math.colostate.edu/~tavener/andre/DD_multiphysics.pdf)

#### Set path and parameters

In [170]:
import numpy as np
import math
%run DD_Utilities.ipynb
%run MS_Utilities.ipynb
iprint= 4

In [171]:
iparam= 5

In [172]:
if iparam == 5:
    n= 12
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,12] #List of ends (inclusive) or integer representing overlap
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [173]:
if iparam == 0:
    n= 10
    p= 4 
    b= [1,2,4,7]
    e= [3,4,8,10] #List of ends (inclusive) or integer representing overlap
    K= 3
    if K > 500:
        raise ValueError("K is too large")

In [174]:
# This example shows how to create domains that each overlap by 1 column
#small system
if iparam == 1:
    n= 8
    p= 3 
    b= [1,3,5]
    e= 1
    K= 3
    if K > 500:
        raise ValueError("K is too large")

In [175]:
# This example shows the optional parameters for set_parameters
if iparam == 2:
    n= 12
    p= -1 #p should be a positive integer or -1
    b= [-1] # an evenly dispersed set of domains will be chosen
    e= [-1] #List of ends or integer representing overlap or [-1] to automaically choose overlap
    K= 6
    if K > 500:
        raise ValueError("K is too large")

In [176]:
#subdomains contained in others
if iparam == 3:
    n= 10
    p= 5 #p should be a positive integer or -1
    b= [1,1,5,5,7]
    e= [2,4,6,8,10] 
    K= 6
    if K > 500:
        raise ValueError("K is too large")

In [177]:
#overlap does not connect all subdomains
if iparam == 4:
    n= 9
    p= 4 
    b= [1,3,6,8]
    e= [2,5,7,9]
    K= 3
    if K > 500:
        raise ValueError("K is too large")

In [178]:
print(p,b,e)
nn,nb,ne,index= fix_domains(n,p,b,e,iprint)

4 [1, 2, 5, 9] [3, 7, 10, 12]


#### Establish and solve forward system 

In [179]:
sread= True    #Read system of equations from iparam[iparam].csv
swrite= False  #Write system to iparam[iparam].csv
mpert= 0.02

M,x,y,xstore= Linear_system(n,sread,iparam)
wpert= Make_wpert(n,p,K,sread,iparam)
if iprint >= 4:
    print(M)
    print('y')
    print(y)        
    print('x')
    print(x)

if swrite:
    save_to_csv(n,p,K,0,nb,nn,ne,M,y,mpert,wpert,iparam)

[[1.24170220e+01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
  3.96767474e-01 5.38816734e-01 4.19194514e-01 6.85219500e-01]
 [2.04452250e-01 1.28781174e+01 2.73875932e-02 6.70467510e-01
  4.17304802e-01 5.58689828e-01 1.40386939e-01 1.98101489e-01
  8.00744569e-01 9.68261576e-01 3.13424178e-01 6.92322616e-01]
 [8.76389152e-01 8.94606664e-01 1.20850442e+01 3.90547832e-02
  1.69830420e-01 8.78142503e-01 9.83468338e-02 4.21107625e-01
  9.57889530e-01 5.33165285e-01 6.91877114e-01 3.15515631e-01]
 [6.86500928e-01 8.34625672e-01 1.82882773e-02 1.27501443e+01
  9.88861089e-01 7.48165654e-01 2.80443992e-01 7.89279328e-01
  1.03226007e-01 4.47893526e-01 9.08595503e-01 2.93614148e-01]
 [2.87775339e-01 1.30028572e-01 1.93669579e-02 6.78835533e-01
  1.22116281e+01 2.65546659e-01 4.91573159e-01 5.33625451e-02
  5.74117605e-01 1.46728575e-01 5.89305537e-01 6.99758360e-01]
 [1.02334429e-01 4.14055988e-01 6.94400158e-01 4.14179270e-01
  4

### Iterative strategy
$u^{\{k\}}= u^{\{k-1\}} + Br = u^{\{k-1\}} + B(f-Au^{\{k-1\}})$, $\quad k=1, \dots, K,$ where $B$ is an approximation to $A^{-1}$

<ins>Multiplicative<ins>
$A_i= R_i A R_i^\top \in \mathbb{R}^{m_i \times m_i}, \\
B_i= R_i^\top A_i^{-1} R_i  = R_i^\top ( R_i A R_i^\top)^{-1} R_i  \in \mathbb{R}^{n \times n}, \\
C_i= B_i A   \in \mathbb{R}^{n \times n},  \\
D_i= (I-C_i) \in \mathbb{R}^{n \times n},  \\
f_i= B_i f   \in \mathbb{R}^{n}, \; i=1,\dots,p.
$

In [ ]:
#Construct domains, R,A,B,C and f
R= Rmatrices(n,p,nn,index,iprint)
A= Amatrices(M,R,n,p,nn,iprint)
B= Bmatrices(A,R,n,p,nn,iprint)
C= Cmatrices(M,B,n,p,iprint) 
D= Dmatrices_multiplicative(C,n,p,iprint)
f= fvector(B,y,n,p,iprint)

#### Perform $K$  iterations computing subdomain contributions independently
$
u^{\{k+i/p\}} = u^{\{k+(i-1)/p\}} + B_i(f-Au^{\{k+(i-1)/p\}}) =D_i u^{\{k+(i-1)/p\}} + f_i \quad i=1,\dots,p$

In [ ]:
#Perform D iteration
vstore= Diteration_multiplicative(D,f,n,p,K,iprint)
if iprint >= 4:
    print('vstore')
    print(vstore)


#### Perform $K$ iterations using iteration matrices $S$ and $T$,  and  which compute the solution in all $p$ domains simultanously
Let $p=5$ and construct S and T, g and v such that

$S = \pmatrix{
 I   &  0   &   0   &  0   &   0   \cr
-D_2 &  I   &   0   &  0   &   0   \cr
 0   & -D_3 &   I   &  0   &   0   \cr
 0   &   0  &  -D_4 &  I   &   0   \cr
 0   &   0  &   0   & -D_5 &   I
} \in \mathbb{R}^{np \times np}$,
$T =
\pmatrix{
 0 & 0 & 0 & 0 & -D_1  \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0
} \in \mathbb{R}^{np \times np}$, 
$g =
\pmatrix{
f_1  \cr
f_2  \cr
f_3  \cr
f_4  \cr
f_5
}
\in \mathbb{R}^{np}$,
$v^{\{k\}} =
\pmatrix{
u^{\{(k-1)+1/5\}}  \cr
u^{\{(k-1)+2/5\}}  \cr
u^{\{(k-1)+3/5\}}  \cr
u^{\{(k-1)+4/5\}}  \cr
u^{\{k\}}
}
\in \mathbb{R}^{np}$,

In [ ]:
# Construct S, T and g and solve
S= Smatrix_multiplicative(D,n,p,iprint)
T= Tmatrix_multiplicative(D,n,p,iprint)
g= gvector_multiplicative(f,n,p,iprint)

wstore= STiteration(S,T,g,n,p,K,iprint)
if iprint >= 4:
    print('wstore')
    print(wstore)

#### Perform $K$ iterations as single matrix solve

$K=6$ iterations of multiplicative Schwarz can be written as the $npK$-dimensional system of equations $Uz=h$, where

$U=
\pmatrix{
 S & 0 & 0 & 0 & 0 & 0   \cr
 T & S & 0 & 0 & 0 & 0   \cr
 0 & T & S & 0 & 0 & 0   \cr
 0 & 0 & T & S & 0 & 0   \cr
 0 & 0 & 0 & T & S & 0   \cr
 0 & 0 & 0 & 0 & T & S
}
$
,
$z=
\pmatrix{
v^{\{1\}}  \cr
v^{\{2\}}  \cr
v^{\{3\}}  \cr
v^{\{4\}}  \cr
v^{\{5\}}  \cr
v^{\{6\}}
}
$
,
$
h=
\pmatrix{
g  \cr
g  \cr
g  \cr
g  \cr
g  \cr
g
}
$
.

In [ ]:
# Construct U and h and solve
U= Umatrix_multiplicative(S,T,n,p,K,iprint) 
h= hvector_multiplicative(g,n,p,K,iprint)

# Solve using U and h
zstore= Uhsolve_multiplicative(U,h,n,p,K,iprint);
if iprint >= 4:
    print('zstore')
    print(zstore)

#### Compare Solutions from all three methods

In [ ]:
solution_compare(xstore,vstore,wstore,zstore,iprint)

### Add discretization error and compute error estimates and effectivity ratios

#### $K$  iterations computing subdomain contributions independently

In [ ]:
# Set adjoint RHS
psi= np.ones((n,1))

# Perform D iteration with error: add random error to each subdomain solve
v_store,va_store,r_store= Diteration_approx(D,f,n,p,K,iprint,mpert,wpert)
if iprint >= 6:
    print(v_store)
    print(va_store)
    print(rstore)

#### Calculate the discretization error

In [ ]:
# Perform adjoint solve 
phi_store= Diteration_adjoint(D,psi,n,p,K,iprint)

# Compute error estimate as (r,phi)
qoi_discretization_error_estimate= r_store.T @ phi_store

# Compute error and error in QoI directly
vesoln=  v_store[n*p*K + n*(p-1):n*p*(K+1),0:1]
vsoln=  va_store[n*p*K + n*(p-1):n*p*(K+1),0:1]
verror= vesoln - vsoln

qoi_discretization_error= psi.T @ verror

#### Calculate the effectivity ratio

In [ ]:
effectivity_ratio_disc = qoi_discretization_error_estimate / qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}')
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

#### Calculate the total error

In [ ]:
total_error= xstore - vsoln
qoi_total_error= psi.T @ total_error

phi= np.linalg.solve(M.T,psi)
r= y - M @ vsoln
qoi_total_error_estimate= r.T @ phi

#### Calculate the effectivity ratio

In [ ]:
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

#### Calculate the iteration error

In [ ]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

#### $K$ iterations as single matrix solve

In [ ]:
# Perform U iteration with error: add random error to linear system solution
zexact,zapprox,zresid= Uhsolve_multiplicative_approx(U,h,n,p,K,iprint,mpert,wpert)

# Adjoint data (Adjoint system is of size npK)
psi_npK= np.zeros((n*p*K,1))
psi_npK[n*p*K - n:n*p*K,0]= psi.reshape(n)

#  Solve adjoint equation
phi_npK= np.linalg.solve(U.T,psi_npK)
if iprint >= 2:
    print(f'Norm of difference between phi_npK and phi_store = {np.linalg.norm(phi_npK-phi_store):13.6e}')

# Compute error estimate
qoi_discretization_error_estimate= phi_npK.T @ zresid

# Compute error and error in QoI directly
xsoln= zexact[n*p*K - n:n*p*K,0:1]
zsoln= zapprox[n*p*K - n:n*p*K,0:1]
zerror= xsoln-zsoln
qoi_discretization_error= psi.T @ zerror

#### Calculate the effectivity ratio

In [ ]:
effectivity_ratio_disc= qoi_discretization_error_estimate/qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}' )
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

#### Calculate the total error

In [ ]:
total_error= xstore - zsoln
qoi_total_error= psi.T @ total_error

r= y - M@zsoln
qoi_total_error_estimate= r.T @ phi

#### Calculate the effectivity ratio

In [ ]:
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

#### Calculate iteration error

In [ ]:
qoi_iteration_error= qoi_total_error - qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')